In [1]:
from skimage.io import imread
import numpy as np
import fnmatch
import os
import re
import random
import time

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import backend as K
from keras.backend.tensorflow_backend import set_session
from keras.utils import Sequence, to_categorical

from keras.applications.densenet import DenseNet121



Using TensorFlow backend.


In [2]:
#TODO check shuffle
class CustomDataGenerator(Sequence):

    def __init__(self, directory, batch_size, classes):
        self.directory = directory
        self.batch_size = batch_size
        self.classes = classes
        
        #create filenames, labels
        dirs = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
        print(dirs)
        self.files = []
        for d in dirs:
            fullpath_dir = os.path.join(directory,d)
            new_elements = [(f,d) for f in os.listdir(fullpath_dir)]
            self.files.extend(new_elements)
        
        # shuffle
        random.shuffle(self.files)
        print(len(self.files))
    
    
    def getLabels(self, offsetStr, viewStr, altitudeStr): 
        offset = 0
        view = 0
        altitude = 0
        for i, e in enumerate(self.classes[0]):
            if e == offsetStr:
                offset = i
                break
            
        for i, e in enumerate(self.classes[1]):
            if e == viewStr:
                view = i
                break
        
        for i, e in enumerate(self.classes[2]):
            if e == altitudeStr:
                altitude = i
                break
                
        return offset, view, altitude
    
    
    def __len__(self):
        return int(np.ceil(len(self.files) / float(self.batch_size)))

    def on_epoch_end(self):
        'Shuffle for the next epoch'
        random.shuffle(self.files)
    
    def __getitem__(self, idx):
        batch_x = []
        batch_y1 = []
        batch_y2 = []
        batch_y3 = []
        batch = self.files[idx * self.batch_size : (idx + 1) * self.batch_size]
        #print(self.directory)
        #print ('files from getitem(): ', batch)
        p = re.compile('(left|center|right)Offset_(left|straight|right)View_(low|medium|high)Altitude')
        
        for (file, d) in batch:
            
            finds = p.findall(d)
            #TODO assert only finds has only 1 element [(_,_)]
            (offsetStr, viewStr, altitudeStr) =  finds[0]
            (offset, view, altitude) = self.getLabels(offsetStr, viewStr, altitudeStr)
            
            folder = os.path.join(self.directory, d)
            rgba = imread(os.path.join(folder, file))
            rgb = rgba[:,:,:3]
            
            input_array = rgb / 255.0
            
            batch_x.append(input_array)
            batch_y1.append(offset)
            batch_y2.append(view)
            batch_y3.append(altitude)
         
        return np.array(batch_x), [np.array(to_categorical(batch_y1, 3)), np.array(to_categorical(batch_y2, 3)), np.array(to_categorical(batch_y3, 3))]


In [3]:
#sanity test CustomDataGenerator
# directory = directory = "E:\\UAV_drone\\data\\training\\lateral_and_view_normal\\train"
# classes = [['left', 'center', 'right'], ['left', 'straight', 'right']]
# datagen = CustomDataGenerator(directory, 5, classes)
# datagen.__getitem__(0)


In [4]:
config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.6
config.gpu_options.allow_growth = True  #dynamically grow the memory used on the GPU
set_session(tf.Session(config=config))

# dimensions of our images.
img_width, img_height = 512, 512

train_data_dir = 'E:\\UAV_drone\\data\\training\\constant_distance_to_lines\\train'
val_data_dir = 'E:\\UAV_drone\\data\\training\\constant_distance_to_lines\\val'
nb_train_samples = 118646
nb_validation_samples = 29802
epochs = 100
batch_size = 8
img_channels = 3
nb_classes = 3

if K.image_data_format() == 'channels_first':
    input_shape = (img_channels, img_width, img_height)
else:
    input_shape = (img_width, img_height, img_channels)

# this is the augmentation configuration we will use for training
classes = [['left', 'center', 'right'], ['left', 'straight', 'right'], ['low', 'medium', 'high']]
train_datagen = CustomDataGenerator(train_data_dir, batch_size, classes)

# this is the augmentation configuration we will use for testing:
val_datagen = CustomDataGenerator(val_data_dir, batch_size, classes)


# checkpoint
current_directory = os.getcwd()
checkpoint_dir = os.path.join(current_directory, 'checkpoints')
result_dir = os.path.join(current_directory, 'results')

if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

file="model_densenet121_view-offset-altitude_{epoch:02d}.hdf5"
checkpoint =  ModelCheckpoint(os.path.join(checkpoint_dir, file), verbose=1)

NAME = "densenet121_view-offset-altitude_{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME), write_graph = True, write_images=True)


['centerOffset_leftView_highAltitude', 'centerOffset_leftView_lowAltitude', 'centerOffset_leftView_mediumAltitude', 'centerOffset_rightView_highAltitude', 'centerOffset_rightView_lowAltitude', 'centerOffset_rightView_mediumAltitude', 'centerOffset_straightView_highAltitude', 'centerOffset_straightView_lowAltitude', 'centerOffset_straightView_mediumAltitude', 'leftOffset_leftView_mediumAltitude', 'leftOffset_rightView_mediumAltitude', 'leftOffset_straightView_highAltitude', 'leftOffset_straightView_lowAltitude', 'leftOffset_straightView_mediumAltitude', 'rightOffset_leftView_mediumAltitude', 'rightOffset_rightView_mediumAltitude', 'rightOffset_straightView_highAltitude', 'rightOffset_straightView_lowAltitude', 'rightOffset_straightView_mediumAltitude']
118646
['centerOffset_leftView_highAltitude', 'centerOffset_leftView_lowAltitude', 'centerOffset_leftView_mediumAltitude', 'centerOffset_rightView_highAltitude', 'centerOffset_rightView_lowAltitude', 'centerOffset_rightView_mediumAltitude

In [5]:
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model 

net = DenseNet121(input_shape=input_shape, include_top=False, weights=None, classes=nb_classes)
x = net.output

outputs = []
multi_output = 3
for i in range(multi_output):
    pool = GlobalAveragePooling2D()(x)
    dense = Dense(nb_classes, activation='softmax')(pool)
    outputs.append(dense)
    
model = Model(net.inputs, outputs=outputs, name='densenet')

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [6]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 518, 518, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 256, 256, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 256, 256, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [7]:
model.fit_generator(
    train_datagen,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_datagen,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[checkpoint, tensorboard])



Epoch 1/100
14830/14830 [==============================] - 9994s 674ms/step - loss: 2.3901 - dense_1_loss: 0.8785 - dense_2_loss: 0.6291 - dense_3_loss: 0.8824 - dense_1_acc: 0.5729 - dense_2_acc: 0.7049 - dense_3_acc: 0.5601 - val_loss: 1.0993 - val_dense_1_loss: 0.6617 - val_dense_2_loss: 0.1992 - val_dense_3_loss: 0.2384 - val_dense_1_acc: 0.7036 - val_dense_2_acc: 0.9294 - val_dense_3_acc: 0.9113

Epoch 00001: saving model to C:\Users\hoang\WorkingSpace\TrainingModels\keras\checkpoints\model_densenet121_view-offset-altitude_01.hdf5
Epoch 2/100
14830/14830 [==============================] - 9901s 668ms/step - loss: 0.4081 - dense_1_loss: 0.2168 - dense_2_loss: 0.0680 - dense_3_loss: 0.1233 - dense_1_acc: 0.9208 - dense_2_acc: 0.9791 - dense_3_acc: 0.9536 - val_loss: 0.5551 - val_dense_1_loss: 0.3342 - val_dense_2_loss: 0.1254 - val_dense_3_loss: 0.0955 - val_dense_1_acc: 0.8757 - val_dense_2_acc: 0.9537 - val_dense_3_acc: 0.9628

Epoch 00002: saving model to C:\Users\hoang\WorkingSp

KeyboardInterrupt: 

## continue training if necessary

In [ ]:
# from keras.models import load_model
# model_file = 'model-uav-view_offset_altitude-normal-newlossfunction-41.hdf5'
# model_full_path = os.path.join(checkpoint_dir, model_file)
# model = load_model(model_full_path)

# file="model-uav-view_offset_altitude-normal-newlossfunction-moredata-{epoch:02d}.hdf5"
# checkpoint =  ModelCheckpoint(os.path.join(checkpoint_dir, file), verbose=1)

# model.fit_generator(
#     train_datagen,
#     steps_per_epoch=nb_train_samples // batch_size,
#     epochs=epochs,
#     validation_data=val_datagen,
#     validation_steps=nb_validation_samples // batch_size,
#     callbacks=[checkpoint])